In [1]:
from data_preprocessing import loadData,getScaledAndOneHotEncoderedX,getLineFromFile,simpleScale,testModelOnData,scaleWithFeaturesAndKeepLocation
from data_preprocessing import checkNegative
from data_preprocessing import decisionTreeDemo

In [32]:
def randomForestDemo(X, y, TX, Ty):
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier()
    testModelOnData(model, X, y, TX, Ty)


In [3]:
def extraTreeDemo(X, y, TX, Ty):
    from sklearn.ensemble import ExtraTreesClassifier
    # model = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
    model = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=2,
                                 max_features=4, random_state=0)
    testModelOnData(model, X, y, TX, Ty)

In [4]:
X, y, TX, Ty = loadData()

scaledAndOneHotX = getScaledAndOneHotEncoderedX(X) 
scaledAndOneHotTX = getScaledAndOneHotEncoderedX(TX) 
print(scaledAndOneHotX.shape)

32561 15
16281 15
(32561, 105)


In [5]:
continuous_features = list(map(lambda x: x - 1, [1, 3, 5, 11, 12, 13]))
scaledX = scaleWithFeaturesAndKeepLocation(X, continuous_features)
scaledTX = scaleWithFeaturesAndKeepLocation(TX, continuous_features)
print(X.shape, scaledX.shape)

X12 = X[:3]
scaledX12 = scaledX[:3]
print(X12[:,continuous_features])
print(scaledX12[:,continuous_features])


(32561, 14) (32561, 14)
[[  3.90000000e+01   7.75160000e+04   1.30000000e+01   2.17400000e+03
    0.00000000e+00   4.00000000e+01]
 [  5.00000000e+01   8.33110000e+04   1.30000000e+01   0.00000000e+00
    0.00000000e+00   1.30000000e+01]
 [  3.80000000e+01   2.15646000e+05   9.00000000e+00   0.00000000e+00
    0.00000000e+00   4.00000000e+01]]
[[ 0.03067056 -1.06361075  1.13473876  0.1484529  -0.21665953 -0.03542945]
 [ 0.83710898 -1.008707    1.13473876 -0.14592048 -0.21665953 -2.22215312]
 [-0.04264203  0.2450785  -0.42005962 -0.14592048 -0.21665953 -0.03542945]]


In [6]:
'''
[All decision trees use np.float32 arrays internally. If training data is not in this format, a copy of the dataset will be made.](http://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use)
Do not see difference.
'''

import numpy as np
X = X.astype(np.float32)
TX = TX.astype(np.float32)
scaledX = scaledX.astype(np.float32)
scaledTX = scaledTX.astype(np.float32)
scaledAndOneHotX = scaledAndOneHotX.astype(np.float32)
scaledAndOneHotTX = scaledAndOneHotTX.astype(np.float32)





In [7]:
feature_names = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country']
class_names = ["gt 50K","le 50K"]
print(len(feature_names))

14


In [8]:
'''
Usage:
    saveTree2DotAndPdf(clf, feature_names=feature_names, class_names=class_names)
'''
def saveTree2DotAndPdf(clf, feature_names, class_names, file_name='tmp', showImage=False):
    from sklearn import tree
    dotFile = file_name+".dot"
    pdfFile = file_name+".pdf"
    with open(dotFile, 'w') as f:
        f = tree.export_graphviz(clf, out_file=f,
                            feature_names=feature_names,
                            class_names=class_names,
                            filled=True, 
                            rounded=True,  
                            special_characters=True
                            )
    print('save to dot file done\n')
    import pydotplus 
    # dot_data = tree.export_graphviz(clf, out_file=None) 
    # graph = pydotplus.graph_from_dot_data(dot_data) 
    graph = pydotplus.graph_from_dot_file(dotFile)
    graph.write_pdf(pdfFile)
    print('save to pdf file done\n')

    
    if showImage:
        print('prepring to show the image, may take a long time...\n')
        from IPython.display import Image
        Image(graph.create_png())
        
        

In [9]:
# Default decisionTree
decisionTreeDemo(X, y, TX, Ty)
decisionTreeDemo(scaledX, y, scaledTX, Ty)
decisionTreeDemo(scaledAndOneHotX,y,scaledAndOneHotTX,Ty)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      7841
        1.0       1.00      1.00      1.00     24720

avg / total       1.00      1.00      1.00     32561

[[ 7841     0]
 [    1 24719]]


             precision    recall  f1-score   support

        0.0       0.60      0.62      0.61      3846
        1.0       0.88      0.87      0.88     12435

avg / total       0.82      0.81      0.81     16281

[[ 2372  1474]
 [ 1562 10873]]


[ 0.80818132  0.8133407   0.81258638]
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_s

In [10]:
# [1.10.5. Tips on practical use](http://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use)

#1 feature selection/dimensionality reduction 

'''
## 1.1 SelectKBest
Test different k,found when k=2, we got best scores.
The parameters selected is [10,11], aka, 'capital-gain' and 'capital-loss'
'''

from sklearn.feature_selection import SelectKBest,chi2
skb = SelectKBest(chi2, k=2).fit(X, y) 

X_new = skb.transform(X)
TX_new = skb.transform(TX)


print(X.shape)
print(X_new.shape,TX_new.shape)
print(skb.get_support(indices=True))

decisionTreeDemo(X_new, y, TX_new, Ty)



(32561, 14)
(32561, 2) (16281, 2)
[10 11]
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
             precision    recall  f1-score   support

        0.0       0.97      0.31      0.47      7841
        1.0       0.82      1.00      0.90     24720

avg / total       0.86      0.83      0.80     32561

[[ 2450  5391]
 [   75 24645]]


             precision    recall  f1-score   support

        0.0       0.97      0.31      0.47      3846
        1.0       0.82      1.00      0.90     12435

avg / total       0.86      0.83      0.80     16281

[[ 1184  2662]
 [   36 12399]]


[ 0.83287267  0.83287267  0.82861882]


In [11]:
'''
# Select K best with scaled data

No imporvement
'''

from sklearn.feature_selection import SelectKBest,chi2
skb = SelectKBest(chi2, k=2).fit(X, y) 

checkNegative(scaledX)

scaledX_new = skb.transform(scaledX)
scaledTX_new = skb.transform(scaledTX)


print(X.shape)
print(scaledX_new.shape,scaledTX_new.shape)
print(skb.get_support(indices=True))



decisionTreeDemo(scaledX_new, y, scaledTX_new, Ty)



'''
chi2 failed because of it [Compute chi-squared stats between each non-negative feature and class](https://github.com/scikit-learn/scikit-learn/blob/14031f6/sklearn/feature_selection/univariate_selection.py#L305)
'''

print(scaledAndOneHotX.shape)
# checkNegative(scaledAndOneHotX)

from sklearn.feature_selection import SelectKBest,chi2,f_classif

skb2 = SelectKBest(f_classif, k=50).fit(scaledAndOneHotX, y)
scaledAndOneHotXKBest = skb2.transform(scaledAndOneHotX)
scaledAndOneHotTXKBest = skb2.transform(scaledAndOneHotTX)

decisionTreeDemo(scaledAndOneHotXKBest, y, scaledAndOneHotTXKBest, Ty)


Enter ckeck negative..
141817
(0, 2)
-1.06361
(32561, 14)
(32561, 2) (16281, 2)
[10 11]
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
             precision    recall  f1-score   support

        0.0       0.97      0.31      0.47      7841
        1.0       0.82      1.00      0.90     24720

avg / total       0.86      0.83      0.80     32561

[[ 2450  5391]
 [   75 24645]]


             precision    recall  f1-score   support

        0.0       0.92      0.24      0.39      3846
        1.0       0.81      0.99      0.89     12435

avg / total       0.84      0.82      0.77     16281

[[  939  2907]
 [   84 12351]]


[ 0.83287267  0.83287267  0.82861882]
(32561, 105)
DecisionTreeClassifier(class_weight=None, criterion='

In [30]:
'''
## 1.2 Tree-based feature selection

It seems that we found nice attrs to use, but the default tree is not so good.
'''

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

clf = ExtraTreesClassifier()
clf = clf.fit(X, y)
print(clf.feature_importances_)

model = SelectFromModel(clf, prefit=True)
extraTreesX = model.transform(X)
extraTreesTX = model.transform(TX)
print(extraTreesX.shape, extraTreesTX.shape)
print(extraTreesX[1])

decisionTreeDemo(extraTreesX, y, extraTreesTX, Ty)



# scaled data
clf = clf.fit(X, y)
print(clf.feature_importances_)

model = SelectFromModel(clf, prefit=True)
extraTreesScaledX = model.transform(scaledX)
extraTreesScaledTX = model.transform(scaledTX)
print(scaledX.shape, scaledTX.shape, extraTreesScaledX.shape, extraTreesScaledTX.shape)
print(extraTreesScaledX[1])

decisionTreeDemo(extraTreesScaledX, y, extraTreesScaledTX, Ty)



# scaled and onehot encodered data
clf.fit(scaledAndOneHotX, y)
model = SelectFromModel(clf, prefit=True)
extraTreeScaledAndOneHotX = model.transform(scaledAndOneHotX)
extraTreeScaledAndOneHotTX = model.transform(scaledAndOneHotTX)
print(scaledAndOneHotX.shape, scaledAndOneHotTX.shape, extraTreeScaledAndOneHotX.shape, extraTreeScaledAndOneHotTX.shape)

decisionTreeDemo(extraTreeScaledAndOneHotX, y, extraTreeScaledAndOneHotTX, Ty)

[ 0.15265516  0.04189486  0.16356164  0.03056916  0.08820702  0.08197178
  0.07991579  0.0711433   0.01323944  0.03709689  0.09349672  0.03161799
  0.09370832  0.02092191]
(32561, 7) (16281, 7)
[  5.00000000e+01   8.33110000e+04   1.30000000e+01   0.00000000e+00
   4.00000000e+00   0.00000000e+00   1.30000000e+01]
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      7841
        1.0       1.00      1.00      1.00     24720

avg / total       1.00      1.00      1.00     32561

[[ 7841     0]
 [    4 24716]]


             precision    recall  f1-score   support

        0.0       0.58      0.59      0.58      3846
        1.0      

In [28]:
'''
## 1.3 PCA way

Use the extraTree got nums as the n_components, aka, X->7, scalaedX->7, scaledAndOneHotX->19

params:
    >batch_size,The number of samples to use for each batch. 
     If batch_size is None, then batch_size is inferred from the data and set to 5 * n_features. 
     
     Not suitable for our onehot encodered data.

'''

from sklearn.decomposition import PCA, IncrementalPCA

n_components = 7
ipca = IncrementalPCA(n_components=n_components, batch_size=None)
ipca.fit(X, y)
pcaX = ipca.transform(X)
pcaTX = ipca.transform(TX)

print(pcaX.shape, pcaTX.shape)

n_components = 7
ipca = IncrementalPCA(n_components=n_components, batch_size=None)
pcaScaledX = ipca.fit_transform(scaledX)
pcaScaledTX = ipca.fit_transform(scaledTX)

print(pcaScaledX.shape, pcaScaledTX.shape)


n_components = 19
ipca2 = IncrementalPCA(n_components=n_components, batch_size=50)
pcaScaledAndOneHotX = ipca2.fit_transform(scaledAndOneHotX)
pcaScaledAndOneHotTX = ipca2.fit_transform(scaledAndOneHotTX)

print(pcaScaledAndOneHotX.shape, pcaScaledAndOneHotTX.shape)





(32561, 7) (16281, 7)
(32561, 7) (16281, 7)
(32561, 19) (16281, 19)


In [31]:
'''
Tweak the params to get better performance by hand.

1.Only change the max_depth, [5-10] gives similar nice results of 0.84, and train/test datasets seems alike.

2.[Balance your dataset before training to prevent the tree from being biased toward the classes that are dominant.](http://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use)
class_weight="balanced", see [class_weight : dict, list of dicts, “balanced” or None, optional (default=None)](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)
     However, not so appealling.

3.
'''

from sklearn import tree
model = tree.DecisionTreeClassifier(max_depth=8)
# testModelOnData(model, X, y, TX, Ty)

# testModelOnData(model, extraTreesX, y, extraTreesTX, Ty)
testModelOnData(model, extraTreesScaledX, y, extraTreesScaledTX, Ty)
# testModelOnData(model, extraTreeScaledAndOneHotX, y, extraTreeScaledAndOneHotTX, Ty)

# testModelOnData(model, pcaX, y, pcaTX, Ty)
# testModelOnData(model, pcaScaledX, y, pcaScaledTX, Ty)
# testModelOnData(model, pcaScaledAndOneHotX, y, pcaScaledAndOneHotTX, Ty)


# saveTree2DotAndPdf(model, feature_names=feature_names, class_names=class_names, file_name='maxdepth4_extraTree')

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
             precision    recall  f1-score   support

        0.0       0.75      0.61      0.67      7841
        1.0       0.88      0.93      0.91     24720

avg / total       0.85      0.86      0.85     32561

[[ 4748  3093]
 [ 1618 23102]]


             precision    recall  f1-score   support

        0.0       0.72      0.57      0.64      3846
        1.0       0.88      0.93      0.90     12435

avg / total       0.84      0.85      0.84     16281

[[ 2202  1644]
 [  858 11577]]


[ 0.84613967  0.8462318   0.85257532]


In [42]:

'''
Tweak RandomForest params to get better performance by hand.

1.max_features
    [1.11.2.3. Parameters](http://scikit-learn.org/stable/modules/ensemble.html#parameters)
    max_features=sqrt(n_features) for classification tasks
'''

from sklearn.ensemble import RandomForestClassifier
from math import sqrt,ceil

'''
# baseline

They seems nice, but overfit, should have better results.
'''
# randomForestDemo(X, y, TX, Ty) # [ 0.84374424  0.84208587  0.84630978]
# randomForestDemo(extraTreesX, y, extraTreesTX, Ty) # [ 0.83176709  0.83407039  0.83654289]
# randomForestDemo(extraTreesScaledX, y, extraTreesScaledTX, Ty) # [ 0.83103004  0.84051962  0.83165945]
# randomForestDemo(extraTreeScaledAndOneHotX, y, extraTreeScaledAndOneHotTX, Ty) # [ 0.82881887  0.83250415  0.83433152]

max_features = ceil(sqrt(X.shape[1]))
print(max_features)
model = RandomForestClassifier(n_estimators=20, max_depth=None,max_features=max_features,
    min_samples_split=2, random_state=None)


testModelOnData(model, X, y, TX, Ty)
# testModelOnData(model, extraTreesX, y, extraTreesTX, Ty)
# testModelOnData(model, extraTreesScaledX, y, extraTreesScaledTX, Ty)

max_features2 = ceil(sqrt(extraTreeScaledAndOneHotX.shape[1]))
print(max_features2)
model2 = RandomForestClassifier(n_estimators=20, max_depth=None,max_features=max_features2,
    min_samples_split=2, random_state=None)
testModelOnData(model2, extraTreeScaledAndOneHotX, y, extraTreeScaledAndOneHotTX, Ty)

# testModelOnData(model, pcaX, y, pcaTX, Ty)
# testModelOnData(model, pcaScaledX, y, pcaScaledTX, Ty)
# testModelOnData(model, pcaScaledAndOneHotX, y, pcaScaledAndOneHotTX, Ty)


# saveTree2DotAndPdf(model, feature_names=feature_names, class_names=class_names, file_name='maxdepth4_extraTree')

4
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=4, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
             precision    recall  f1-score   support

        0.0       1.00      0.99      0.99      7841
        1.0       1.00      1.00      1.00     24720

avg / total       1.00      1.00      1.00     32561

[[ 7795    46]
 [   36 24684]]


             precision    recall  f1-score   support

        0.0       0.70      0.64      0.67      3846
        1.0       0.89      0.91      0.90     12435

avg / total       0.84      0.85      0.85     16281

[[ 2451  1395]
 [ 1070 11365]]


[ 0.84927216  0.85074627  0.85487884]
5
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
        